In [1]:
#pip install ortools

In [66]:
from ortools.sat.python import cp_model
import pandas as pd

# Initialize the CP-SAT model
model = cp_model.CpModel()

# Parameters
num_teams = 8
teams = range(1, num_teams + 1)
num_rounds = 20
rounds = range(1, num_rounds + 1)
rinks = ['A', 'B', 'C', 'D']  # Rinks
weeks = [(round - 1) // 2 + 1 for round in rounds]  # Weeks calculation for convenience

# Create variables for each game, including the potential rink it's played on
game_vars = {}
for round in rounds:
    for team1 in teams:
        if team1 < num_teams:  # Ensure unique pairings
            for team2 in range(team1 + 1, num_teams + 1):
                for rink in rinks:
                    game_vars[(round, team1, team2, rink)] = model.NewBoolVar(
                        f'game_r{round}_t{team1}_t{team2}_r{rink}')

# Ensure each team plays exactly one game per round, but not necessarily limiting rink assignment here
for round in rounds:
    for team in teams:
        model.Add(sum(game_vars[round, min(team, other), max(team, other), rink]
                      for other in teams if team != other for rink in rinks) == 1)

# Each rink hosts exactly one game per round
for round in rounds:
    for rink in rinks:
        model.Add(sum(game_vars[round, team1, team2, rink]
                      for team1 in teams for team2 in teams if team1 < team2) == 1)

# Enforce the constraint that teams don't play on the same rink twice in the same week
for week in set(weeks):
    for team in teams:
        for rink in rinks:
            model.Add(sum(game_vars[round, min(team, other), max(team, other), rink]
                          for round in rounds if (round - 1) // 2 + 1 == week
                          for other in teams if team != other) <= 1)

# Assuming each team plays each other team no more than 3 times across all rinks
# Note: This constraint may need adjustment based on league rules and the number of teams and weeks
for team1 in teams:
    for team2 in teams:
        if team1 < team2:
            model.Add(sum(game_vars[round, team1, team2, rink] for round in rounds for rink in rinks) <= 3)

# Minimum gap between games for the same teams - optional based on specific league rules
min_gap = 6
for team1 in teams:
    for team2 in teams:
        if team1 < team2:
            for round in range(1, num_rounds - min_gap + 1):
                model.Add(sum(game_vars[r, team1, team2, rink] for r in range(round, min(round + min_gap, num_rounds + 1)) for rink in rinks) <= 1)

# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Prepare to populate the DataFrame
schedule_data = []

# Check if a solution exists
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for round in rounds:
        for team1 in teams:
            for team2 in teams:
                if team1 < team2:
                    for rink in rinks:
                        if solver.Value(game_vars[(round, team1, team2, rink)]) == 1:
                            week = (round - 1) // 2 + 1
                            schedule_data.append({
                                'Week': week,
                                'Round': round,
                                'Rink': rink,
                                'Team 1': team1,
                                'Team 2': team2
                            })

# Convert the schedule data to a DataFrame
schedule_df = pd.DataFrame(schedule_data)

# Order the DataFrame by week, round, and rink
schedule_df = schedule_df.sort_values(by=['Week', 'Round', 'Rink']).reset_index(drop=True)

# Display the DataFrame for the first week as an example
print(schedule_df[schedule_df['Week'] == 1])


   Week  Round Rink  Team 1  Team 2
0     1      1    A       2       3
1     1      1    B       4       5
2     1      1    C       1       6
3     1      1    D       7       8
4     1      2    A       1       5
5     1      2    B       3       7
6     1      2    C       4       8
7     1      2    D       2       6


In [67]:
# Save DataFrame to CSV without index
schedule_df.to_csv("schedule.csv", index=False)

In [70]:
# Load the CSV file into a DataFrame
schedule_df = pd.read_csv('schedule.csv')

# Display the first few rows to understand its structure
schedule_df.head()

,Week,Round,Rink,Team 1,Team 2
0,1,1,A,2,3
1,1,1,B,4,5
2,1,1,C,1,6
3,1,1,D,7,8
4,1,2,A,1,5


In [71]:
def find_exceptions(schedule_df):
    """
    Identifies exceptions where the same team is scheduled in the same rink for both rounds in the same week.
    
    Parameters:
    - schedule_df: A pandas DataFrame with columns ['Round', 'Week', 'Rink', 'Team 1', 'Team 2'].
    
    Returns:
    - exceptions_df: A pandas DataFrame with columns ['Week', 'Rink', 'Team'] listing all exceptions.
    """
    exceptions = []

    # Iterate over the weeks
    for week in schedule_df['Week'].unique():
        week_df = schedule_df[schedule_df['Week'] == week]

        # Iterate over the rinks
        for rink in rinks:
            rink_df = week_df[week_df['Rink'] == rink]

            # Check if a team plays in the same rink in both rounds of the week
            for team in range(1, num_teams + 1):
                if len(rink_df[(rink_df['Team 1'] == team) | (rink_df['Team 2'] == team)]) > 1:
                    # If a team is found in the same rink in both rounds, add to exceptions
                    exceptions.append({
                        'Week': week,
                        'Rink': rink,
                        'Team': team
                    })

    # Convert exceptions to a DataFrame and return it
    exceptions_df = pd.DataFrame(exceptions)
    return exceptions_df

# Example usage:
# First, ensure you have the schedule_df DataFrame created and populated as before
exceptions_df = find_exceptions(schedule_df)

if not exceptions_df.empty:
    print("Exceptions found:")
    print(exceptions_df)
else:
    print("No exceptions found.")


No exceptions found.
